# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

사람들이 LLM(언어 모델)을 사용하여 만드는 가장 일반적이고 복잡한 응용 중 하나는 문서에 대한 질문을 답하거나 그 문서를 기반으로 질문에 답하는 시스템입니다. 예를 들어, PDF 파일이나 웹페이지 또는 회사의 내부 문서 컬렉션에서 추출된 텍스트가 주어졌을 때, LLM을 사용하여 해당 문서의 내용에 관한 질문에 답할 수 있습니까? 이것은 사용자가 더 깊은 이해를 얻고 필요한 정보에 접근하는데 도움이 됩니다. 이것은 정말 강력합니다. 왜냐하면 이러한 언어 모델들을 원래 학습되지 않았던 데이터와 결합하기 시작하기 때문입니다. 그래서 그들은 당신의 사용 사례에 훨씬 더 유연하고 적응할 수 있게 만듭니다.

Andrew가 언급했듯이, 이것은 우리가 가진 체인 중에서 가장 인기 있는 것 중 하나입니다. 사실 임베딩과 벡터 저장소는 현대 기술 중 가장 강력한 것들 중 일부이므로 아직 보지 못했다면 배우는 것이 매우 가치 있습니다.

In [1]:
#pip install --upgrade langchain

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [5]:
from langchain.indexes import VectorstoreIndexCreator

체인 구축 시 도움이 될 몇 가지를 가져옵니다. 우리는 검색 QA 체인을 가져올 것입니다. 이 체인은 일부 문서 위에서 검색 작업을 수행합니다.

OpenAI 언어 모델 챗봇도 가져올 겁니다.

다음으로, 언어 모델과 결합할 프라이버시 데이터를 로드하는 데 사용될 문서 로더도 필요합니다.
여기선 CSV 파일로 제공됩니다.
그래서 CSV 로더도 함께 가져옵니다.

마지막으로 벡터 저장소도 가져와야 합니다.
벡터 저장소 종류는 여러 가지가 있으며 나중에 자세히 설명하겠지만 "DocArrayInMemorySearch" 벡터 저장소로 시작해 보겠습니다.

이것은 메모리 내 벡터 저장소로, 외부 데이터베이스에 연결할 필요가 없어 시작하기 쉽습니다.

또한 Jupyter 노트북에서 정보를 표시하는 데 사용되는 display와 markdown을 가져옵니다.

우리는 아웃도어 의류에 대한 CSV 파일을 제공하며, 이를 언어 모델과 결합하여 사용하게 됩니다. 여기서는 이 파일의 경로를 가진 CSV 로더를 초기화합니다.

In [6]:
#pip install docarray

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

다음으로 "VectorStoreIndexCreator"라는 인덱스를 가져옵니다. 이것은 매우 쉽게 벡터 저장소를 만드는 데 도움이 됩니다. 아래에서 볼 수 있듯이, 이 작업을 수행하는데 몇 줄의 코드만 필요합니다.

In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
response = index.query(query)

생성하기 위해 우리는 두 가지를 지정할 것입니다.
첫째, 벡터 저장소 클래스를 지정합니다.
앞서 언급했듯이, 우리는 이 벡터 저장소를 사용할 것입니다. 왜냐하면 특히 초보자가 시작하기 쉽기 때문입니다.
그런 다음 "from_loaders"라고 부르며 문서 로더 목록을 받아들입니다.
우리가 실제로 신경 쓰는 로더가 하나밖에 없으므로 그것을 여기에 전달합니다.

그럼 이제 생성되었고 우리는 그에 대해 질문을 할 수 있습니다.
아래에서 정확히 어떤 일이 일어나고 있는지 알아볼 것이므로 지금은 걱정하지 마십시오. 여기서 질의문으로 시작하겠습니다.
그런 다음 "index.query"를 사용하여 응답을 생성하고 이 질의문을 전달합니다.

다시 말하지만, 아래에서 내부 작동 원리에 대해 알아볼 것입니다.

지금은 그저 응답하는 것을 기다립니다.
완료된 후 반환된 내용이 정확히 무엇인지 확인해 보겠습니다.
태양 차단 기능이 있는 모든 셔츠의 이름과 설명으로 구성된 마크다운 형식의 표와 함께 되돌아옵니다.

In [10]:
display(Markdown(response))



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

In [11]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [12]:
docs = loader.load()

In [13]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [15]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [16]:
print(len(embed))

1536


In [17]:
print(embed[:5])

[-0.021900920197367668, 0.006746490020304918, -0.018175246194005013, -0.039119575172662735, -0.014097143895924091]


In [18]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [23]:
retriever = db.as_retriever()

In [24]:
llm = ChatOpenAI(temperature = 0.0)


In [25]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [26]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [27]:
display(Markdown(response))

| Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Recommended by The Skin Cancer Foundation. |
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front/back cape venting. Made with 52% polyester and 48% nylon. |
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and front/back cape venting. Made with 71% nylon and 29% polyester. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, front/back cape venting, and two front bellows pockets. Made with 100% polyester. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are made with high-performance fabrics that are moisture-wicking, wrinkle-resistant, and abrasion-resistant. The Men's Plaid Tropic Shirt and Men's Tropical Plaid Short-Sleeve Shirt both have front/back cape venting for added breathability. The Sun Shield Shirt is recommended by The Skin Cancer Foundation.

In [28]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [29]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [30]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
display(Markdown(response))

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is machine washable and dryable. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 255 | Sun Shield Shirt | This shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is handwashable and line dry. It is rated UPF 50+ for superior protection from the sun's UV rays. It is abrasion-resistant and wicks moisture for quick-drying comfort. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+.

The Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is abrasion-resistant and wicks moisture for quick-drying comfort. It is rated UPF 50+ for superior protection from the sun's UV rays. It is handwashable and line dry.

In [32]:
response = index.query(query, llm=llm)

In [33]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])